<a href="https://colab.research.google.com/github/swarnalathasiva18/Generate-Q-A-using-GPT/blob/main/NLP_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install transformers


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
torch.cuda.is_available()

True

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Step 1: Data Preprocessing

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews.csv')
df.dropna(inplace=True)
df.shape



(4632, 12)

In [22]:
def to_sentiment(rating):
    rating = int(rating)
    if rating == 1 or rating == 2:
        return 0.0
    else:
        return 1.0

# Apply to the dataset
df['sentiment'] = df["score"].apply(to_sentiment)
df.columns
df = df[['content','sentiment']]
df.head()



KeyError: ignored

In [20]:
#fine tuning Bert

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=1)
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
# Tokenization for training

encoded_data = df_train['content'].apply(lambda x: tokenizer.encode_plus(
    x,
    add_special_tokens=True,
    max_length=256,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
))

input_ids = torch.cat([e['input_ids'] for e in encoded_data], dim=0)
attention_masks = torch.cat([e['attention_mask'] for e in encoded_data], dim=0)
labels = torch.tensor(df_train['sentiment'].map({'Positive': 1, 'Negative': 0}).values)

labels.shape

torch.Size([3500])

In [29]:
# Model Architecture

dataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
# Training

batch_size = 16
num_epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device).float()
        labels = batch[2].to(device).float()

        optimizer.zero_grad()
        outputs = model(input_ids, attention_masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [ ]:
# Tokenization for testing

encoded_data = df_test['content'].apply(lambda x: tokenizer.encode_plus(
    x,
    add_special_tokens=True,
    max_length=256,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
))

input_ids = torch.cat([e['input_ids'] for e in encoded_data], dim=0)
attention_masks = torch.cat([e['attention_mask'] for e in encoded_data], dim=0)
labels = torch.tensor(df_test['sentiment'].map({'Positive': 1, 'Negative': 0}).values)

labels.shape

dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
model.eval()
with torch.no_grad():
  for batch in dataloader:
    input_ids = input_ids.to(device)
    attention_masks = attenetion_masks.to(device)
    labels = labels.to(device)

    outputs = model(input_ids, attention_masks)
    predicted_labels = np.argmax(outputs.logits.cpu().numpy(), axis=1)

accuracy = (predicted_labels == labels.cpu().numpy()).mean()
print(f'Accuracy: {accuracy}')

Accuracy: 0.0
